This notebook has been used for the initial development and testing of the basic components of the Retrieval Augmented Generation (RAG) system. It includes various developmental stages from prototyping initial concepts to refining the functionalities of the RAG components. The code and insights gained from this notebook were instrumental in building a more structured and deployable version of the RAG system, which is encapsulated in the Call_RAG.py script.

# Setup

Instal

In [1]:
#! pip install pypdf
#! pip install langchain
#! pip install "langchain[docarray]"
#! pip install chromadb

Import

In [1]:
import os
import openai
import sys
from langchain_community.document_loaders import PyPDFLoader

# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

API KEY

In [2]:
sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]

# Load document

In [21]:
loaders = [
    PyPDFLoader("../pdf/EdAP 2020_EN.pdf"),
    PyPDFLoader("../pdf/SOF book-web-rev3d-hires.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [22]:
len(docs)

878

In [25]:
page = docs[870]
print(f"Contenu de la page \n{page.page_content[0:500]}")
print(f"METADATA : {page.metadata}")

Contenu de la page 
440   |  Les forêts du bassin du Congo
BibliographieRoca T, Letouzé E. 2016. La révolution des données est-elle en marche ? Implications pour la statistique 
publique et la démocratie. Afrique contemporaine. 258(2):95-111.
Rosen GE, Smith KF. 2010. Summarizing the evidence on the international trade in illegal wildlife. 
EcoHealth. 7(1):24-32.
RRI (Rights and Resources Initiative). 2017. Securing community land rights : Priorities and Opportunities to 
advance climate and sustainable development
METADATA : {'source': '../pdf/SOF book-web-rev3d-hires.pdf', 'page': 466}


# Split document

In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=50, separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
splits = text_splitter.split_documents(docs)
print(len(splits))
print(len(docs))

2196
878


# Embed / Vector store

Setup Embedding and Vector store

In [44]:
embedding = OpenAIEmbeddings()
persist_directory = "../vectorstore/chroma/"


# Delete the persist_directory if you want to force the generatation of another vector store
#! rm -rf persist_directory

Generate new Vectore Store if foler Chroma is empty

In [45]:
# Check if the vector store already exists
if os.path.exists(persist_directory):
    # If the vector store exists, load it
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
else:
    # If the vector store does not exist, generate it
    # Assuming 'splits' is a list of documents already defined elsewhere in your notebook
    vectordb = Chroma.from_documents(
        documents=splits, embedding=embedding, persist_directory=persist_directory
    )

Similarity search

In [49]:
# print(vectordb._collection.count())
question = "What is OFAC stand for ?"
answer_docs = vectordb.similarity_search(question, k=3)
len(answer_docs)
answer_docs[0].page_content
for d in answer_docs:
    print(d.metadata)

{'page': 158, 'source': '../pdf/EdAP 2020_EN.pdf'}
{'page': 216, 'source': '../pdf/SOF book-web-rev3d-hires.pdf'}
{'page': 387, 'source': '../pdf/EdAP 2020_EN.pdf'}


MMR search

In [50]:
answer_docs = vectordb.max_marginal_relevance_search(question, k=2, fetch_k=3)
len(answer_docs)
answer_docs[0].page_content
for d in answer_docs:
    print(d.metadata)

{'page': 158, 'source': '../pdf/EdAP 2020_EN.pdf'}
{'page': 387, 'source': '../pdf/EdAP 2020_EN.pdf'}


Filter on SOF (State of forest) only (doesn't work)

In [56]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of EdAP 2020_EN.pdf or SOF book-web-rev3d-hires.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the document",
        type="integer",
    ),
]

document_content_description = "Report of forest and protected area"
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectordb, document_content_description, metadata_field_info, verbose=True
)

In [59]:
question = "what OFAC stand for, check answer in SOF document"
docs = retriever.get_relevant_documents(question)
for d in docs:
    print(d.metadata)

[]


Compression of retrieved data

In [64]:
# Guillaume comment :
# In this section, we will use a Langchain compressor to reduce the amount of text retrieve.
# By doing this, the summarized text produced is almost an answer


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )


# Wrap our vectorstore
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")

compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vectordb.as_retriever()
)

question = "what is the difference between OFAC and COMIFAC?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/home/guillaume/.local/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/guillaume/.local/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/guillaume/.local/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/guillaume/.local/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

OFAC and COMIFAC
----------------------------------------------------------------------------------------------------
Document 2:

- COMIFAC
- Central Africa
- sustainable and coordinated management of forest ecosystems
- orientation, harmonization and monitoring of forestry and environmental policies
- emerged from the commitments made in March 1999 by the Heads of State of Central Africa in the “Yaoundé Declaration”
- ten member countries of the subregion
- common natural heritage
- legal framework governed by the February 2005 treaty: “Treaty on the Conservation and Sustainable Management of Forest Ecosystems in Central Africa and to establish the Central African Forests Commission”
- Convergence Plan defines the shared ten-year intervention strategies of Central African States and development partners in the field of conservation and sustainable management of forest and savanna ecosystems
- second edition of this plan, covering the period 2015-2025
- Web site: www.comi

# Question answering

Stuff technics : all document are sent to the LLM to answer

In [71]:
# Here we are creating a answer chain with Langchain. We make a specific prompt.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [73]:
question = "What is the difference between OFAC and COMIFAC?"
result = qa_chain.invoke({"query": question})
result["result"]

"OFAC is a specialized unit of COMIFAC responsible for coordinating the Forest Observatory and disseminating information on Central Africa's forests and ecosystems. COMIFAC, on the other hand, is an organization responsible for harmonizing and monitoring forestry and environmental policies in Central Africa. Thanks for asking!"

Map reduce : it combine all retrieved data before sending to the LLM to answer

In [78]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever(), chain_type="map_reduce"
)
question = (
    "What is the main expertise of OFAC ? How it contribute to reduce deforestation ?"
)
result = qa_chain.invoke({"query": question})
result["result"]

"OFAC's main expertise is in collecting and managing environmental data in Central Africa to support the sustainable management of forest ecosystems. It contributes to reducing deforestation by providing essential tools for steering and sharing knowledge for better governance and sustainable management of forest ecosystems in the region. Thanks for asking!"

Refine : it improve the answer with each different document

In [80]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever(), chain_type="refine"
)
question = (
    "What is the main expertise of OFAC ? How it contribute to reduce deforestation ?"
)
result = qa_chain.invoke({"query": question})
result["result"]

"OFAC's main expertise is in collecting and managing environmental data in Central Africa to support the sustainable management of forest ecosystems. It contributes to reducing deforestation by providing essential tools for steering and sharing knowledge for better governance and sustainable management of forest ecosystems in the region. Thanks for asking!"

# Q/A with memory

In [81]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

question = (
    "What is the main expertise of OFAC ? How it contribute to reduce deforestation ?"
)
result = qa.invoke({"question": question})

In [82]:
result["answer"]

'The main expertise of OFAC (Central Africa Forest Observatory) lies in ensuring the availability of information to support the sustainable management of forest ecosystems in Central Africa. OFAC contributes to reducing deforestation by collecting and managing environmental data at different scales, conducting annual campaigns to collect reference data in its member states, and collaborating with various partners to harmonize and disseminate information. Additionally, OFAC plays a crucial role in promoting knowledge transfer and skills between countries and actors, ultimately providing essential tools for steering and sharing knowledge for better governance and sustainable management of forest ecosystems in the region.'

In [83]:
question = "Which difference with COMIFAC ?"  # Check if it understand it's difference between OFAC and COMIFAC
result = qa.invoke({"question": question})
result["answer"]

'OFAC and COMIFAC are both mentioned in the provided context, but the text does not explicitly outline the specific differences between the two entities. The text describes OFAC as playing a role in data storage, analysis, and transmission to support decision-making within the reach of managers, while COMIFAC is mentioned in the context of a Council of Ministers and holders of global issues. For more detailed differences between OFAC and COMIFAC, further information or sources specific to these organizations would be needed.'